<a href="https://colab.research.google.com/github/avisruti/AI-ML/blob/main/MNIST_week9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sys
import torch
import torch.nn as nn
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.autograd import Variable
import torch.nn.functional as F
import torch.optim as optim

In [2]:
def load_subset(full_train_set, full_test_set, label_one, label_two):
    # Sample the correct train labels
    train_set = []
    data_lim = 20000
    for data in full_train_set:
        if data_lim>0:
            data_lim-=1
            if data[1]==label_one or data[1]==label_two:
                train_set.append(data)
        else:
            break

    test_set = []
    data_lim = 1000
    for data in full_test_set:
        if data_lim>0:
            data_lim-=1
            if data[1]==label_one or data[1]==label_two:
                test_set.append(data)
        else:
            break

    return train_set, test_set

In [3]:
def train(model,optimizer,train_loader,epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        if torch.cuda.is_available():
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()

In [4]:
def test(model,test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        if torch.cuda.is_available():
            data, target = data.cuda(), target.cuda()
        with torch.no_grad():
            data, target = Variable(data), Variable(target)
        output = model(data)
        test_loss += F.cross_entropy(output, target, reduction='sum').item()#size_average=False
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.data.view_as(pred)).long().cpu().sum()

    test_loss /= len(test_loader.dataset)
    acc=100. * float(correct.to(torch.device('cpu')).numpy())
    test_accuracy = (acc / len(test_loader.dataset))
    return test_accuracy

In [5]:
class AlexNet(nn.Module):
    def __init__(self, num=10):
        super(AlexNet, self).__init__()
        self.feature = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=32, kernel_size=5, stride=1, padding=1), # 28x28 -> 26x26
            nn.ReLU(),
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1), # 26x26 -> 26x26
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2), # 26x26 -> 13x13
            nn.Conv2d(in_channels=64, out_channels=96, kernel_size=3, stride=1, padding=1), # 13x13 -> 13x13
            nn.ReLU(),
            nn.Conv2d(in_channels=96, out_channels=64, kernel_size=3, stride=1, padding=1), # 13x13 -> 13x13
            nn.ReLU(),
            nn.Conv2d(in_channels=64, out_channels=32, kernel_size=3, stride=1, padding=1), # 13x13 -> 13x13
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=1)  # 13x13 -> 12x12
        )

        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(32 * 12 * 12, 2048),  # 32 channels * 12x12 feature map size
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(2048, 1024),
            nn.ReLU(),
            nn.Linear(1024, num)  # Output layer, num features = 10
        )

    def forward(self, x):
        x = self.feature(x)
        x = x.view(x.size(0), -1)  # Flatten the tensor
        x = self.classifier(x)
        return x

In [14]:
    input_data_one = 8
    input_data_two = 7
    epochs = 5

    """  Call to function that will perform the computation. """
    #if input_data_one.isdigit() and input_data_two.isdigit() and epochs.isdigit():

    label_one = int(input_data_one)
    label_two = int(input_data_two)
    epochs = int(epochs)

    if label_one!=label_two and 0<=label_one<=9 and 0<=label_two<=9:
            torch.manual_seed(42)
            # Load MNIST dataset
            trans = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (1.0,))])
            full_train_set = dset.MNIST(root='./data', train=True, transform=trans, download=True)
            full_test_set = dset.MNIST(root='./data', train=False, transform=trans)
            batch_size = 16
            # Get final train and test sets
            train_set, test_set = load_subset(full_train_set,full_test_set,label_one,label_two)

            train_loader = torch.utils.data.DataLoader(dataset=train_set,batch_size=batch_size,shuffle=False)
            test_loader = torch.utils.data.DataLoader(dataset=test_set,batch_size=batch_size,shuffle=False)

            model = AlexNet()
            if torch.cuda.is_available():
                model.cuda()

            optimizer = optim.SGD(model.parameters(), lr=0.01)

            for epoch in range(1, epochs+1):
                train(model,optimizer,train_loader,epoch)
                accuracy = test(model,test_loader)

            print(round(accuracy,2))


    else:
           print("Invalid input")


95.21
